In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
D1 = ["bird","cat","cat","dog","dog","bird","tiger","tiger"]
D2 = ["cat","tiger","cat","dog"]
D3 = ["dog","bird","bird","cat"]
D4 = ["cat","tiger","cat","dog"]
D5 = ["tiger","tiger","dog","tiger"]
D6 = ["cat","cat","tiger","tiger"]
D7 = ["bird","cat","bird"]
D8 = ["dog","cat","bird"]
D9 = ["cat","dog","tiger"]
D10 = ["tiger","tiger","tiger"]
D11 = ["bird"]*10 + ["cat"]*9 + ["dog"]*8 + ["tiger"]*7
D12 = ["bird"]*23 + ["cat"]*11 + ["dog"]*8 + ["tiger"]*1
D13 = ["bird"]*0 + ["cat"]*0 + ["dog"]*11 + ["tiger"]*25
D14 = ["bird"]*1 + ["cat"]*2 + ["dog"]*4 + ["tiger"]*3
D15 = ["bird"]*8 + ["cat"]*11 + ["dog"]*3 + ["tiger"]*1
words = ["bird","cat","dog","tiger"]
documents = ["D1","D2","D3","D4","D5","D6","D7","D8","D9","D10","D11","D12","D13","D14","D15"]

In [45]:
documents_matrix = np.array([])
for i in range(1,16):
    document = "D" + str(i)
    document_array = eval(document)
    document_array = np.array(document_array)
    document_matrix = np.zeros(len(words))
    for j in range(len(words)):
        document_matrix[j] = np.sum(document_array == words[j])
    if i == 1:
        documents_matrix = document_matrix
    else:
        documents_matrix = np.vstack((documents_matrix, document_matrix))
document_matrix = documents_matrix.transpose().astype(int)
docoment_matrix_pd = pd.DataFrame(document_matrix, index = words, columns = documents)
docoment_matrix_pd

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
bird,2,0,2,0,0,0,2,1,0,0,10,23,0,1,8
cat,2,2,1,2,0,2,1,1,1,0,9,11,0,2,11
dog,2,1,1,1,1,0,0,1,1,0,8,8,11,4,3
tiger,2,1,0,1,3,2,0,0,1,3,7,1,25,3,1


In [46]:
def latentSemanticIndexing(documents_matrix):
    # Calculate the SVD of the matrix
    U, S, V = np.linalg.svd(documents_matrix)

    # Calculate the reduced matrix
    k = 2
    S = np.diag(S)
    S = S[:k, :k]
    U = U[:, :k]
    V = V[:k, :]
    # print(f'original matrix shape: {documents_matrix}')
    # print(f'U : ', U)
    # print(f'S : ', S)
    # print(f'V : ', V)
    
    reduced_matrix = np.dot(np.dot(U, S), V)
    return reduced_matrix

In [47]:
# Assuming latentSemanticIndexing returns a 2D array
documents = latentSemanticIndexing(documents_matrix).transpose()

# Convert to DataFrame
documents_pd = pd.DataFrame(documents)

# Print the shape of the documents array for debugging
print(documents.shape)

# Assuming 'words' is a list of words corresponding to the rows of the original matrix
# and 'document_names' is a list of names for the documents
document_names = [f'Doc{i+1}' for i in range(documents.shape[1])]

# Set DataFrame columns and index
documents_pd.columns = document_names
documents_pd.index = words

# Print the DataFrame for debugging
documents_pd

(4, 15)


,Doc1,Doc2,Doc3,Doc4,Doc5,Doc6,Doc7,Doc8,Doc9,Doc10,Doc11,Doc12,Doc13,Doc14,Doc15
bird,2.314066,0.932663,1.913147,0.932663,-0.130826,0.640894,1.727026,1.262682,0.506568,-0.316947,11.088938,21.031072,-0.593896,1.930193,10.343489
cat,1.631637,0.670634,1.272720,0.670634,0.047201,0.500209,1.133102,0.846625,0.389723,-0.092418,7.690458,13.976358,0.765655,1.453972,6.886831
dog,1.806159,0.856577,0.739986,0.856577,1.275769,0.977665,0.511860,0.553865,0.716959,1.047642,7.387280,7.990801,11.239741,2.425505,4.058359
tiger,2.106514,1.128100,0.107110,1.128100,2.870708,1.619384,-0.242104,0.212759,1.158906,2.521494,7.343456,0.865421,24.853802,3.751089,0.704084


In [48]:
query = []
nums_bird = 7
nums_cat = 5
nums_dog = 4
nums_tiger = 3
query = ["bird"] * nums_bird + ["cat"] * nums_cat + ["dog"] * nums_dog + ["tiger"] * nums_tiger
# Create a matrix of the query
query_matrix = np.zeros((1, len(words)))
for word in query:
    if word in words:
        query_matrix[0, words.index(word)] += 1
query_matrix = query_matrix.transpose().astype(int)
query_pd = pd.DataFrame(query_matrix, index = words, columns = ["query"])
query_pd

,query
bird,7
cat,5
dog,4
tiger,3


### Cosine Similarities

In [49]:
cosine_similarities = np.dot(query_matrix.transpose(), documents) / (np.linalg.norm(query_matrix) * np.linalg.norm(documents))
cosine_similarities = cosine_similarities.flatten()
cosine_similarities_pd = pd.DataFrame(cosine_similarities, index = document_names, columns = ["cosine similarity"])
cosine_similarities_pd = cosine_similarities_pd.sort_values(by = "cosine similarity", ascending = False)
print(f'Cosine similarities:\n\r{cosine_similarities_pd}')
print(f'Best document: {cosine_similarities_pd.index[0]}')

Cosine similarities:
       cosine similarity
Doc12           0.566351
Doc11           0.377301
Doc15           0.281724
Doc13           0.268237
Doc14           0.093927
Doc1            0.085295
Doc3            0.051844
Doc7            0.042930
Doc4            0.037566
Doc2            0.037566
Doc8            0.035840
Doc6            0.035459
Doc5            0.029336
Doc9            0.026644
Doc10           0.020422
Best document: Doc12


### Euclidean Distances

In [50]:
euclidean_distances = []
query_vector = query_matrix.flatten()
print(f'Query vector: {query_vector}')
for i in range(documents.shape[1]):
    document_vector = documents[:, i]
    euclidean_distance = np.linalg.norm(query_vector - document_vector)
    euclidean_distances.append(euclidean_distance)

euclidean_distances_pd = pd.DataFrame(euclidean_distances, index = document_names, columns = ["euclidean distance"])
euclidean_distances_pd = euclidean_distances_pd.sort_values(by = "euclidean distance", ascending = True)
print(f'Euclidean distances:\n\r{euclidean_distances_pd}')
print(f'Best document: {euclidean_distances_pd.index[0]}')


Query vector: [7 5 4 3]
Euclidean distances:
       euclidean distance
Doc15            4.473665
Doc1             6.238197
Doc14            6.428097
Doc11            7.368667
Doc3             7.665846
Doc7             8.089222
Doc2             8.303078
Doc4             8.303078
Doc8             8.355352
Doc6             8.469177
Doc9             8.808366
Doc5             9.100388
Doc10            9.402968
Doc12           17.260617
Doc13           24.608931
Best document: Doc15
